In [38]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import os

import matplotlib.pyplot as plt
%matplotlib inline

# autoreload modules
%load_ext autoreload
%autoreload 2

# set seeds for reproducibility
np.random.seed(12)
tf.set_random_seed(12)

Loaded backend module://ipykernel.pylab.backend_inline version unknown.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import env
from train_vaegan import train_vaegan

In [45]:
from sacred.observers import FileStorageObserver
from sacred import Experiment

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

In [48]:
cfg = {
    'cfg': {
        "dataset": {
            "index": "../data/processed/thingi10k_index.csv",
            "tag": "animal",
            "pctile": 0.9
        },
        "model": {
            "voxels_dim": 32,
            "batch_size": 22,
            "example_stl_id": 126660,
            "voxel_prob_threshold": 0.065,
            "enc_lr": 0.0001,
            "dec_lr": 0.0001,
            "dis_lr": 0.0001,
            "keep_prob": 1.0,
            "kl_div_loss_weight": 1,
            "recon_loss_weight": 10000,            
            "latent_dim": 100,
            "verbose": True,
            "debug": True,
            "generator_verbose": False,
            "epochs": 10,
            "input_repeats": 1,
            "display_step": 10,
            "save_step": 10
        }
    }
}

In [ ]:
ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.pctile"
WARNING - root - Added new config entry: "cfg.dataset.tag"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNING - root - Added new config entry: "cfg.model.epochs"
WARNING - root - Added new config entry: "cfg.model.example_stl_id"
WARNING - root - Added new config entry: "cfg.model.generator_verbose"
WARNING - root - Added new config entry: "cfg.model.input_repeats"
WARNING - root - Added new config entry: "cfg.model.keep_prob"
WARNING - root - Added new config entry: "cfg.model.kl_div_loss_weight"
WARNING - root - Added new config entry: "cfg.mo

Epoch: 2, Batch: 0, Elapsed time: 0.62 mins
DEBUG_OP "max clipped_output": 0.5366461277008057
DEBUG_OP "min clipped_output": 0.4565543830394745
DEBUG_OP "mean clipped_output": 0.4992218017578125
	KL Divergence = 3.8757588863372803, Reconstruction Loss = 2023.191162109375, -dis_loss = -87.02186584472656, dec_loss = 78.6535415649414
Epoch: 2, Batch: 1, Elapsed time: 0.66 mins
DEBUG_OP "max clipped_output": 0.5431069731712341
DEBUG_OP "min clipped_output": 0.44493183493614197
DEBUG_OP "mean clipped_output": 0.4990738034248352
	KL Divergence = 5.6454362869262695, Reconstruction Loss = 1864.54345703125, -dis_loss = -91.39228820800781, dec_loss = 82.3212661743164
Epoch: 2, Batch: 2, Elapsed time: 0.71 mins
DEBUG_OP "max clipped_output": 0.5387468338012695
DEBUG_OP "min clipped_output": 0.4589087963104248
DEBUG_OP "mean clipped_output": 0.49884504079818726
	KL Divergence = 3.401291847229004, Reconstruction Loss = 1932.85693359375, -dis_loss = -93.09304809570312, dec_loss = 85.95929718017578
E